<a href="https://colab.research.google.com/github/hamid42643/Toronto-Housing-Price-Prediction/blob/main/JFriedel_Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Assignment 4.1: Making the Cut
Due Monday by 11:59pm Points 90 Submitting a text entry box, a website url, a media recording, or a file upload Attempts 0 Allowed Attempts 3
Introduction
In this assignment, you will leverage k-means clustering to create groupings from the credit default data, experiment with different parameters available
in the model ("k" and distance metrics), and interpret the output from a business perspective.

Instructions
Create a k-means model with the assignment dataset using at least 10 features.
Experiment with at least 3 k values. Be sure to transform variables into the appropriate format before modeling. Note that a larger k will increase the
overhead of interpretation, so it is suggested to keep the k less than 10.
What transformations did you apply to the raw dataset?
What were different k's chosen? What were the differences in the output with those different k's?
Choose a final k that you think reflects the data the best and provide a written interpretation of the different clusters generated by k-means
Why did you choose this k and distance metric?
Why does it appear these groups have been created? What are the influential features?
Are there any inferences you can draw that would be relevant from a business context about the different groups?


In [ ]:
#@title Loading the required libraries

import numpy as np # Pandas for array manipulation
import pandas as pd # Pandas for data manipulation
import seaborn as sns # Seaborn for visualizing. Note: we will also use one of the data sets
import matplotlib.pyplot as plt # Matplotlib for subplots

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler # Import for feature standardization
from sklearn.metrics import silhouette_samples, silhouette_score # For kmeans evaluation

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 500)

In [ ]:
def load_dataset(filename):
    df = pd.read_csv(filename)
    return df
df = load_dataset("train_data.csv")

In [ ]:
from sklearn import tree # Import Tree Classifiers
from sklearn.ensemble import RandomForestClassifier # Import Random Forest Classifiers
from sklearn.model_selection import train_test_split,  RandomizedSearchCV # Import train_test_split function
from sklearn import metrics # Import scikit-learn metrics module for accuracy calculation
from scipy.stats import randint # Generate random numbers
from xgboost import XGBClassifier, plot_importance
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from sklearn.linear_model import LogisticRegression

Data Processing

In [ ]:
shap = df.shape
print("Shape of the dataframe (row, col):",shap,"\r\n")

In [ ]:
df

In [ ]:
# Replace missing values with 0
df = df.fillna(0)

In [ ]:
# Convert enuerations to integer
del df['OCCUPATION_TYPE']
del df['ORGANIZATION_TYPE']
df["HOUSETYPE_MODE"] = df['HOUSETYPE_MODE'].replace({'block of flats': 1, 'terraced house': 2, 'specific housing': 3}).astype(int)
df["FONDKAPREMONT_MODE"] = df['FONDKAPREMONT_MODE'].replace({'reg oper account': 1, 'org spec account': 2, 'reg oper spec account': 3, 'not specified': 4}).astype(int)
df["NAME_HOUSING_TYPE"] = df['NAME_HOUSING_TYPE'].replace({'House / apartment': 1, 'Municipal apartment': 2, 'Office apartment': 3, 'Rented apartment': 4, 'With parents': 5, 'Co-op apartment':6}).astype(int)
df["NAME_EDUCATION_TYPE"] = df['NAME_EDUCATION_TYPE'].replace({'Secondary / secondary special': 1, 'Higher education': 2, 'Lower secondary': 3, 'Incomplete higher': 4, 'Academic degree': 5}).astype(int)
df["NAME_INCOME_TYPE"] = df['NAME_INCOME_TYPE'].replace({'Working': 1, 'Pensioner': 2, 'Commercial associate': 3, 'State servant': 4, 'Unemployed': 5, 'Businessman': 6, 'Student': 7, 'Maternity leave': 7}).astype(int)
df["NAME_TYPE_SUITE"] = df['NAME_TYPE_SUITE'].replace({'Unaccompanied': 1, 'Family': 2, 'Children': 3, 'Spouse, partner': 4, 'Other_A': 5, 'Other_B': 6, 'Group of people': 7}).astype(int)
df["NAME_FAMILY_STATUS"] = df['NAME_FAMILY_STATUS'].replace({'Married': 1, 'Single / not married': 2, 'Civil marriage': 3, 'Widow': 4, 'Separated': 5, 'Unknown': 6, 'Others': 7}).astype(int)
df["WALLSMATERIAL_MODE"] = df['WALLSMATERIAL_MODE'].replace({'Panel': 1, 'Wooden': 2, 'Stone, brick': 3, 'Block': 4, 'Mixed': 5, 'Monolithic': 6, 'Others': 7, 'Unknown': 8}).astype(int)
df["EMERGENCYSTATE_MODE"] = df['EMERGENCYSTATE_MODE'].replace({'No': 1, 'Yes': 2}).astype(int)
df["FLAG_OWN_CAR"] = df['FLAG_OWN_CAR'].replace({'N': 0, 'Y': 1}).astype(int)
df["FLAG_OWN_REALTY"] = df['FLAG_OWN_REALTY'].replace({'N': 0, 'Y': 1}).astype(int)
df["CODE_GENDER"] = df['CODE_GENDER'].replace({'M': 0, 'F': 1}).astype(int)
df["NAME_CONTRACT_TYPE"] = df['NAME_CONTRACT_TYPE'].replace({'Cash loans': 0, 'Revolving loans': 1}).astype(int)
df["WEEKDAY_APPR_PROCESS_START"] = df['WEEKDAY_APPR_PROCESS_START'].replace({'MONDAY': 1, 'TUESDAY': 2, 'WEDNESDAY': 3, 'THURSDAY': 4, 'FRIDAY': 5, 'SATURDAY': 6, 'SUNDAY': 7}).astype(int)
#display the first 5 rows in dataset:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df.head(5)

In [ ]:
df.TARGET

In [ ]:
X=pd.DataFrame(df)

In [ ]:
# Select features
features = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_ANNUITY', 'DAYS_EMPLOYED', 'OWN_CAR_AGE', 'NAME_FAMILY_STATUS']
#features = ['AMT_INCOME_TOTAL']
X_selected = X[features]

In [ ]:
# Select Target
y = X['TARGET']

In [ ]:
target=df.TARGET

All features are continuous values but on different scales. Need to scale data in order to prevent larger magnitude columns from taking over the model. Look at the raw distribution of each feature, and then the means of all features

In [ ]:
means = []
for col in df.columns:
    print(f'Producing stats for column: {col}')
    print(df[col].describe([0.1, 0.25, 0.5, 0.75, 0.9]))
    means.append([col,df[col].describe()['mean']])

Distinct different in mean across the different variables. Need to scale data to accommodate for the k-means algorithm

In [ ]:
pd.DataFrame(means, columns = ['column', 'mean'])

In [ ]:
df.head(5)

In [ ]:
df = df.drop(df.columns[14:122], axis=1)

In [ ]:
shap = df.shape
print("Shape of the dataframe (row, col):",shap,"\r\n")

In [ ]:
#summarize the dataset’s statistical properties, such as count, mean, min, and max:
df.describe()

In [ ]:
# Plot the scaled data

sns.pairplot(df) # Plot each variable against all other variables
plt.show()

In [ ]:
# Let's scale the data

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_data, columns = df.columns)

In [ ]:
# Now let's evaluate the scaled data in the same fashion

scaled_means = []
for col in scaled_df.columns:
    print(f'Producing stats for column: {col}')
    print(scaled_df[col].describe([0.1, 0.25, 0.5, 0.75, 0.9]))
    scaled_means.append([col,scaled_df[col].describe()['mean']])

In [ ]:
pd.DataFrame(scaled_means, columns = ['column', 'mean'])

In [ ]:
# Plot the scaled data
# You can see the variable relationships are maintained but the magnitude of the variables are different relative to the raw data

sns.pairplot(scaled_df) # Plot each variable against all other variables
plt.show()

## K-Means Clustering

Run multiple variations of K-means with different variations of k-means, from 3-6. Evaluate the output - Evaluate by joining the raw data back with the cluster assignments, including the target variable available in the dataset, then looking at how the clusters map to the raw data.

In [ ]:
OMP_NUM_THREADS=1

In [ ]:
def kmeans_execution(df, num_clust):
    # Create models & store labels for different number of clusters
    kmn = KMeans(n_clusters = num_clust, n_init='auto', random_state = 0) # Create KMeans object
    kmn.fit(df) # Apply to the data
    kmn_lbl = kmn.labels_ # Capture K-Means labels
    kmn_distortion = kmn.inertia_ # Used for elbow method
    return kmn, kmn_lbl, kmn_distortion

#kmn_2_mod, kmn_2_labels, kmn_2_dist = kmeans_execution(scaled_df, 2)
#silhouette_2 = silhouette_score(scaled_df, kmn_2_labels)
#print(f'k=2 silhouette average score: {silhouette_2}')
kmn_3_mod, kmn_3_labels, kmn_3_dist = kmeans_execution(scaled_df, 3)
silhouette_3 = silhouette_score(scaled_df, kmn_3_labels)
print(f'k=3 silhouette average score: {silhouette_3}')
#kmn_4_mod, kmn_4_labels, kmn_4_dist = kmeans_execution(scaled_df, 4)
#silhouette_4 = silhouette_score(scaled_df, kmn_4_labels)
#print(f'k=4 silhouette average score: {silhouette_4}')
kmn_5_mod, kmn_5_labels, kmn_5_dist = kmeans_execution(scaled_df, 5)
silhouette_5 = silhouette_score(scaled_df, kmn_5_labels)
print(f'k=5 silhouette average score: {silhouette_5}')
kmn_6_mod, kmn_6_labels, kmn_6_dist = kmeans_execution(scaled_df, 6)
silhouette_6 = silhouette_score(scaled_df, kmn_6_labels)
print(f'k=6 silhouette average score: {silhouette_6}')

In [ ]:
# Elbow Plot - seems like clear "elbow" at k=4

fig = plt.figure(figsize=(15, 5))
#plt.plot(range(2, 7), [kmn_2_dist, kmn_3_dist, kmn_4_dist, kmn_5_dist, kmn_6_dist])
plt.plot(range(3, 6), [kmn_3_dist, kmn_5_dist, kmn_6_dist])
plt.grid(True)
plt.title('Elbow curve')

In [ ]:
# Create evaluation dataset

eval_df = df.copy()
eval_df['target'] = target
eval_df['kmn_3_label'] = kmn_3_labels
#eval_df['kmn_4_label'] = kmn_4_labels
eval_df['kmn_5_label'] = kmn_5_labels
eval_df['kmn_6_label'] = kmn_6_labels

In [ ]:
# Can execute across different values of k to determine differences in clusters
summary_stats, target_groups = group_by_cluster(eval_df, kmn_3_labels)
summary_stats

In [ ]:
# Look at distribution of target variable in different clusters created

target_groups

In [ ]:
OMP_NUM_THREADS=1

In [ ]:
def kmeans_execution(df, num_clust):
    # Create models & store labels for different number of clusters
    kmn = KMeans(n_clusters = num_clust, n_init='auto', random_state = 0) # Create KMeans object
    kmn.fit(df) # Apply to the data
    kmn_lbl = kmn.labels_ # Capture K-Means labels
    kmn_distortion = kmn.inertia_ # Used for elbow method
    return kmn, kmn_lbl, kmn_distortion

#kmn_2_mod, kmn_2_labels, kmn_2_dist = kmeans_execution(scaled_df, 2)
#silhouette_2 = silhouette_score(scaled_df, kmn_2_labels)
#print(f'k=2 silhouette average score: {silhouette_2}')
kmn_3_mod, kmn_3_labels, kmn_3_dist = kmeans_execution(scaled_df, 3)
silhouette_3 = silhouette_score(scaled_df, kmn_3_labels)
print(f'k=3 silhouette average score: {silhouette_3}')
kmn_4_mod, kmn_4_labels, kmn_4_dist = kmeans_execution(scaled_df, 4)
silhouette_4 = silhouette_score(scaled_df, kmn_4_labels)
print(f'k=4 silhouette average score: {silhouette_4}')
kmn_5_mod, kmn_5_labels, kmn_5_dist = kmeans_execution(scaled_df, 5)
silhouette_5 = silhouette_score(scaled_df, kmn_5_labels)
#print(f'k=5 silhouette average score: {silhouette_5}')
#kmn_6_mod, kmn_6_labels, kmn_6_dist = kmeans_execution(scaled_df, 6)
#silhouette_6 = silhouette_score(scaled_df, kmn_6_labels)
#print(f'k=6 silhouette average score: {silhouette_6}')

In [ ]:
# Elbow Plot - seems like clear "elbow" at k=

fig = plt.figure(figsize=(15, 5))
#plt.plot(range(2, 7), [kmn_2_dist, kmn_3_dist, kmn_4_dist, kmn_5_dist, kmn_6_dist])
plt.plot(range(3, 6), [kmn_3_dist, kmn_4_dist, kmn_5_dist])
plt.grid(True)
plt.title('Elbow curve')

In [ ]:
# Create evaluation dataset

eval_df = df.copy()
eval_df['target'] = target
eval_df['kmn_3_label'] = kmn_3_labels
eval_df['kmn_4_label'] = kmn_4_labels
eval_df['kmn_5_label'] = kmn_5_labels
#eval_df['kmn_6_label'] = kmn_6_labels

1)Create a k-means model with the assignment dataset using at least 10 features.
See above.
    
2) Experiment with at least 3 k values. Be sure to transform variables into the appropriate format before modeling.
Note that a larger k will increase the overhead of interpretation, so it is suggested to keep the k less than 10.
    a) What transformations did you apply to the raw dataset?
    Replaced missing data with zeroes.
    Converted enumerations to integers.
    Reduced 122 features down to 14 (removed columns)
    Scaled the remaining columns.
        
    b) What were different k's chosen? What were the differences in the output with those different k's
    I tried 2 different groupings (3,4,6) and (3,4,5).  In the first case I got the highest silhouette score at 3 and the second case I got the
    highest score at 4.  In both cases the graph elbow was at 4.
    
3) Choose a final k that you think reflects the data the best and provide a written interpretation of the different clusters generated by k-means
   
    a) Why did you choose this k and distance metric?
    I believe k = 5 is the best because the highest Means were achieved for Income, Credit and annuity.

    b) Why does it appear these groups have been created?
    I think these three features show the ability to take and handle debt; more so than the other ~120 features.
        
    c) What are the influential features?
    I determined the influencial features ti be income, profession, loan size, employment status, marital status, propery ownership, and credit rating.
        
    d) Are there any inferences you can draw that would be relevant from a business context about the different groups?
    Out of all the graphical data pairings, only Goods Price vs Credit had a cleanly linear relationship.  There I think credit is the key indicator
    that a loan officer should focus on.  
